In [1]:
import pandas as pd
data = pd.read_csv('../dataset_customer_churn.csv', sep='^')
data.drop(labels=['A006_REGISTRO_ANS', 'CODIGO_BENEFICIARIO', 'CLIENTE', 'CD_USUARIO', 'CODIGO_FORMA_PGTO_MENSALIDADE', 'A006_NM_PLANO', 'DIAS_ATE_REALIZAR_ALTO_CUSTO', 'CD_ASSOCIADO', 'ESTADO_CIVIL'], axis=1, inplace=True)
is_NAN = data[data.isna().any(axis=1)]
data.drop(is_NAN.index, axis=0, inplace=True)
data.head(20)

,NUM_BENEFICIARIOS_FAMILIA,SITUACAO,IDADE,SEXO,QTDE_DIAS_ATIVO,QTDE_ATENDIMENTOS,QTDE_ATO_COBERTO_EXECUTADO,QTDE_ATO_N_COBERTO_EXECUTADO,REALIZOU_ENDODONTIA_COBERTA,REALIZOU_EXODONTIA_COBERTA,REALIZOU_PROCEDIMEN_ALTO_CUSTO,FORMA_PGTO_MENSALIDADE,PLANO
0,1.0,DESATIVADO,40.0,F,277.0,1.0,8.0,0.0,NAO,NAO,NAO,debito automatico,408875991
1,2.0,DESATIVADO,56.0,F,1725.0,2.0,10.0,0.0,NAO,NAO,SIM,debito automatico,408875991
2,2.0,DESATIVADO,66.0,F,2006.0,18.0,73.0,8.0,SIM,NAO,SIM,debito automatico,408875991
3,3.0,DESATIVADO,62.0,M,529.0,1.0,1.0,0.0,NAO,NAO,NAO,debito automatico,422427992
4,1.0,DESATIVADO,47.0,M,176.0,0.0,0.0,0.0,NAO,NAO,NAO,boleto,408875991
5,1.0,DESATIVADO,38.0,M,1936.0,0.0,0.0,0.0,NAO,NAO,NAO,debito automatico,703454997
6,4.0,DESATIVADO,50.0,M,1796.0,1.0,14.0,0.0,NAO,NAO,NAO,debito automatico,408875991
7,4.0,DESATIVADO,44.0,M,180.0,5.0,18.0,4.0,NAO,NAO,SIM,debito automatico,408875991
8,1.0,DESATIVADO,39.0,M,1725.0,11.0,30.0,8.0,NAO,NAO,SIM,debito automatico,408875991
9,1.0,DESATIVADO,55.0,F,238.0,3.0,27.0,0.0,NAO,NAO,SIM,debito automatico,408875991


In [2]:
dict_replace = {
    "SIM": 1,
    "NAO": 0,
    'F': 0,
    'M': 1,
    'DESATIVADO': 1,
    'ATIVO': 0,

}
data.replace(dict_replace, inplace=True)
data.drop(labels=182212, axis=0, inplace=True)

In [3]:
data.sort_values(by='IDADE')

,NUM_BENEFICIARIOS_FAMILIA,SITUACAO,IDADE,SEXO,QTDE_DIAS_ATIVO,QTDE_ATENDIMENTOS,QTDE_ATO_COBERTO_EXECUTADO,QTDE_ATO_N_COBERTO_EXECUTADO,REALIZOU_ENDODONTIA_COBERTA,REALIZOU_EXODONTIA_COBERTA,REALIZOU_PROCEDIMEN_ALTO_CUSTO,FORMA_PGTO_MENSALIDADE,PLANO
316545,1.0,0,0.0,1,29.0,0.0,0.0,0.0,0,0,0,boleto,475840164
257962,1.0,0,0.0,1,273.0,0.0,0.0,0.0,0,0,0,boleto,475840164
336719,1.0,0,0.0,0,241.0,0.0,0.0,0.0,0,0,0,boleto,475840164
257959,1.0,0,0.0,0,273.0,0.0,0.0,0.0,0,0,0,boleto,475840164
301569,1.0,0,0.0,0,89.0,0.0,0.0,0.0,0,0,0,boleto,475840164
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85504,1.0,0,108.0,0,1790.0,0.0,0.0,0.0,0,0,0,boleto,475840164
85505,1.0,0,108.0,0,1790.0,0.0,0.0,0.0,0,0,0,boleto,475840164
3679,6.0,1,109.0,1,2883.0,0.0,0.0,0.0,0,0,0,debito automatico,408875991
2486,1.0,1,109.0,0,410.0,2.0,0.0,0.0,0,0,0,debito automatico,422427992


In [40]:
QTDE_DIAS_ATIVO_MENOR_QUE_365 = data['QTDE_DIAS_ATIVO'].loc[data['QTDE_DIAS_ATIVO'] < 365]
data['QTDE_DIAS_ATIVO'] < 365

0          True
1         False
2         False
3         False
4          True
          ...  
386387     True
386388     True
386389     True
386390    False
386391    False
Name: QTDE_DIAS_ATIVO, Length: 386391, dtype: bool

In [47]:
import numpy as np
QTDE_DIAS_ATIVO_MENOR_QUE_365 = np.array(data['QTDE_DIAS_ATIVO'] < 365)
data['QTDE_DIAS_ATIVO_MENOR_QUE_365'] = 0
data.loc[QTDE_DIAS_ATIVO_MENOR_QUE_365,'QTDE_DIAS_ATIVO_MENOR_QUE_365'] = 1
QTDE_DIAS_ATIVO_MENOR_QUE_1000 = np.array((data['QTDE_DIAS_ATIVO'] >= 365) & (data['QTDE_DIAS_ATIVO'] < 1000))
data['QTDE_DIAS_ATIVO_MENOR_QUE_1000'] = 0
data.loc[QTDE_DIAS_ATIVO_MENOR_QUE_1000, 'QTDE_DIAS_ATIVO_MENOR_QUE_1000'] = 1
QTDE_DIAS_ATIVO_MAIOR_QUE_1000 = np.array(data['QTDE_DIAS_ATIVO'] >= 1000)
data['QTDE_DIAS_ATIVO_MAIOR_QUE_1000'] = 0
data.loc[QTDE_DIAS_ATIVO_MAIOR_QUE_1000, 'QTDE_DIAS_ATIVO_MAIOR_QUE_1000'] = 1
data

,NUM_BENEFICIARIOS_FAMILIA,SITUACAO,IDADE,SEXO,QTDE_DIAS_ATIVO,QTDE_ATENDIMENTOS,QTDE_ATO_COBERTO_EXECUTADO,QTDE_ATO_N_COBERTO_EXECUTADO,REALIZOU_ENDODONTIA_COBERTA,REALIZOU_EXODONTIA_COBERTA,REALIZOU_PROCEDIMEN_ALTO_CUSTO,FORMA_PGTO_MENSALIDADE,PLANO,QTDE_DIAS_ATIVO_MENOR_QUE_365,QTDE_DIAS_ATIVO_MENOR_QUE_1000,QTDE_DIAS_ATIVO_MAIOR_QUE_1000
0,1.0,DESATIVADO,40.0,F,277.0,1.0,8.0,0.0,NAO,NAO,NAO,debito automatico,408875991,1,0,0
1,2.0,DESATIVADO,56.0,F,1725.0,2.0,10.0,0.0,NAO,NAO,SIM,debito automatico,408875991,0,0,1
2,2.0,DESATIVADO,66.0,F,2006.0,18.0,73.0,8.0,SIM,NAO,SIM,debito automatico,408875991,0,0,1
3,3.0,DESATIVADO,62.0,M,529.0,1.0,1.0,0.0,NAO,NAO,NAO,debito automatico,422427992,0,1,0
4,1.0,DESATIVADO,47.0,M,176.0,0.0,0.0,0.0,NAO,NAO,NAO,boleto,408875991,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386387,2.0,ATIVO,26.0,F,55.0,2.0,11.0,0.0,NAO,NAO,NAO,boleto digital,481485181,1,0,0
386388,1.0,ATIVO,42.0,M,55.0,1.0,1.0,0.0,NAO,NAO,NAO,boleto digital,481485181,1,0,0
386389,2.0,ATIVO,32.0,M,55.0,3.0,7.0,0.0,SIM,NAO,NAO,boleto digital,481485181,1,0,0
386390,1.0,DESATIVADO,37.0,F,833.0,1.0,5.0,0.0,NAO,NAO,NAO,boleto,408875991,0,1,0


In [ ]:
data_dummified = pd.get_dummies(data)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

x = data_dummified.drop(labels=['SITUACAO'], axis=1)
y = data_dummified['SITUACAO']
np.random.seed(1010)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier(10)
random_forest_classifier.fit(train_x,train_y)
predicts_rfc = random_forest_classifier.predict(test_x)

In [ ]:
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score
def all_scores(predicts, test_y):
    print(f"Accuracy Score: {accuracy_score(test_y,predicts):.3f}\nRecall Score: {recall_score(test_y,predicts):.3f}\nPrecision Score: {precision_score(test_y,predicts):.3f}\nF1 Score: {f1_score(test_y,predicts):.3f}\n")
def scores(validation_results):
    media = validation_results['test_score'].mean()
    desvio_padrao = validation_results['test_score'].std()
    print("Accuracy médio {:.2f}".format(media))
    print("Intervalo [{:.2f}, {:.2f}]".format((media - 2 * desvio_padrao), (media + 2 * desvio_padrao)))


In [ ]:
all_scores(predicts_rfc, test_y)

In [ ]:
from sklearn.model_selection import cross_validate, KFold

validation_results = cross_validate(random_forest_classifier,x,y,cv=KFold(n_splits=10, shuffle=True))
scores(validation_results)
validation_results_df = pd.DataFrame(validation_results)
validation_results_df

In [ ]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV, KFold
tt_x, val_x, tt_y, val_y = train_test_split(x, y, test_size=0.2, stratify=y)
### hiper-parametrização (?)
parametros_para_RSCV = {
    "max_depth" : randint(3, 100),
    "min_samples_split": randint(2, 16),
    "min_samples_leaf": randint(1, 16),
    "bootstrap" : [True, False],
    "criterion": ["gini", "entropy"]
}

RSCross_validation = RandomizedSearchCV(RandomForestClassifier(n_estimators=10), parametros_para_RSCV, n_iter= 16, cv = KFold(n_splits = 5, shuffle=True))
RSCross_validation.fit(tt_x,tt_y)
resultados = pd.DataFrame(RSCross_validation.cv_results_)

In [ ]:
best_random_forest = RSCross_validation.best_estimator_
predicts_rfc = best_random_forest.predict(val_x)
all_scores(predicts_rfc, val_y)

In [ ]:
print(best_random_forest)

In [ ]:
feature_importances = random_forest_classifier.feature_importances_
feature_importances

In [ ]:
from matplotlib import pyplot as plt

plt.pie(feature_importances)

In [ ]:
from sklearn.inspection import permutation_importance
permutation_importance = permutation_importance(best_random_forest, x, y)

In [ ]:
permutation_importance_df = pd.Series(permutation_importance.importances_mean, index=pd.Series([col for col in x.columns]))
permutation_importance_df*100

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
plt.pie(permutation_importance_df)
plt.xlabel("Permutation Importance")

In [ ]:
def scores(model):
    model_predicts = model.predict(test_x)
    return accuracy_score(test_y, model_predicts), recall_score(test_y, model_predicts),precision_score(test_y, model_predicts), f1_score(test_y, model_predicts)

In [ ]:

model_scores = {scores(best_random_forest), scores(random_forest_classifier)}
df_model_scores = pd.DataFrame(model_scores, columns=['Acurracy Score', 'Recall Score', 'Precision Score', 'F1 Score'], index=['BRF', 'RFC'])
df_model_scores